Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# Molecular Classification on IPUs using MolFeat 
## Fine tune pre-trained Transformer models for Blood Brain Barrier Permiability and QM9 prediction 


[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/yoyy6N)
 [![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)


### Introduction: MolFeat on the IPU

The popular [MolFeat Library](https://molfeat.datamol.io) offers a comprehensive open-source collection of pre-trained featurizers for molecules, designed for seamless integration into ML workflows. 
The pre-trained featurizers in the Molfeat library have been trained on large quantities of data from a variety of domains, making them ideally suited to providing an initial featurization on new molecular datasets. This makes it possible to use larger and more sophisticated models even when only very small datasets are available as the fundamental physical properties of the molecule are already represented in the featurizer. 

In this notebook, we present a step-by-step guide on how to employ the Graphcore IPU for fine-tuning a pre-trained Transformer model on the `Blood-Brain Barrier Permeability (BBBP)` dataset from MoleculeNet. The goal is to predict which molecules can cross the blood-brain barrier, an essential property for drug development. Additionally, we demonstrate the versatility of this method for other regression tasks using new datasets.

### Summary Table
|   Domain   |      Tasks       |         Model         |  Datasets  |        Workflow         | Number of IPUs      | Execution Time |
|:----------:|:----------------:|:---------------------:|:----------:|:-----------------------:|:-------------------:|:--------------:|
| Molecules  | Classification / Regression | ChemBERTa-77M-MLM / ChemGPT | BBBP / QM9 | Training, evaluation, inference | Recommended: 4x (Min: 1x) |     5 min     |

### Learning Outcomes
Through this demo, you will acquire the skills to:
- Classify molecules by leveraging a MolFeat featurizer and fine-tuning a Hugging Face Transformer on the IPU.
- Construct an inference workflow for individual molecule predictions.
- Understand how to transition between classification and regression tasks.

### Links to Other Resources
For additional information on MolFeat, consult their [documentation](https://molfeat.datamol.io), and for more details about the datasets employed in this notebook, explore [MoleculeNet](https://moleculenet.org/datasets-1). This notebook presumes prior knowledge of Transformer architecture and PyTorch on the IPU. To review these topics, refer to the relevant tutorials on using PyTorch on the IPU, available [here](https://console.paperspace.com/github/graphcore/Gradient-HuggingFace?machine=Free-IPU-POD4&container=graphcore%2Fpytorch-jupyter%3A3.2.0-ubuntu-20.04-20230331&file=%2Fnatural-language-processing%2Fintroduction_to_optimum_graphcore.ipynb).

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)


## Running on Paperspace

The Paperspace environment lets you run this notebook with almost no set up. To improve your experience we preload datasets and pre-install packages, this can take a few minutes, if you experience errors immediately after starting a session please try restarting the kernel before contacting support. If a problem persists or you want to give us feedback on the content of this notebook, please reach out to through our community of developers using our [slack channel](https://www.graphcore.ai/join-community) or raise a [GitHub issue](https://github.com/graphcore/examples).

Requirements:

* Python packages installed with `pip install -r ./requirements.txt`

In [ ]:
# Make imported python modules automatically reload when the files are changed
# needs to be before the first import.
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install -q -r ./requirements.txt
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

## The Problem...

Let's start by posing a toy problem. Imagine we have a dataset of molecules that we know something about, in this case about the ability of various compounds to penetrate the Blood-Brain-Barrier, but it could be any property we wish. This dataset has been made from experimental results which is ultimately very expensive and time consuming to collect and extend. 

So like any good scientist, we wonder can we take the data we have and create a model to describe the physical results that we can use to make predictions about new molecules where experiments haven't been carried out with reliable results? 

Below we can see an example molecule from the dataset, Propanolol, represented as a SMILES string and visualised in 3D. Looking at the table we can see the expected result. This is the target we are aiming to produce, and by the end of this notebook we will have a model to fill in the rest of the table. 

In [ ]:
from utils import report_molecule_classification

report_molecule_classification(
    "Propanolol", True, None, "[Cl].CC(C)NCC(O)COc1cccc2ccccc12"
)

In [ ]:
import torch
import copy
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from molfeat.trans.pretrained import PretrainedHFTransformer

import poptorch
from torch.utils.data import Dataset
from IPython.display import clear_output
import ipywidgets as widgets


import poptorch

from utils import (
    plot_smoothed_loss,
    report_molecule_classification,
    report_molecule_regression,
)
from utils import Emoji

First we need to select the featurizer and the dataset. In this example we'll use the `ChemBERTa-77M-MLM` pre-trained featurizer and the `BBBP` (Blood Brain Barrier Permiability) dataset. We load the featurizer, read the dataset from the URL provided, then we can look at the dataframe and plot the first molecule from the dataset. 

We can see the dataframe includes the name of the molecule, the label (does the molecule pass through the BBB), and the SMILES string for the molecule. We can then use the utility function `report_molecule_classification` to explore the dataset - look at different molecules in the dataset by changing the `mol_id`. 

In [ ]:
featurizer = PretrainedHFTransformer(
    kind="ChemBERTa-77M-MLM", pooling="bert", preload=True
)

df = pd.read_csv("https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/BBBP.csv")
print(df.head())
print(f"Length of dataset: {len(df)}")

Here the molecule report is showing the basic information - name and target, through this notebook we'll show how to finetune the Transformer model to classify individual molecules with our model in this format. 

In [ ]:
mol_id = 10  # SET THIS VALUE AND RE-RUN THIS CELL TO EXPLORE THE DATASET.

view = report_molecule_classification(
    df.name.values[mol_id], df.p_np.values[mol_id], None, df.smiles.values[mol_id]
)
view.show()

**Exercise:** When you've finished running the notebook, come back here and uncomment this block to run the `ChemGPT-4.7M` model with a regression task. There are a few small changes to make through the notebook for this, but we'll point those out when we get to them. 

In [ ]:
# featurizer = PretrainedHFTransformer(kind="ChemGPT-4.7M", notation="selfies")

# df = pd.read_csv("https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv")
# print(df.head())

# report_molecule_regression(df.mol_id.values[0], df.gap.values[0], None, df.smiles.values[0])

## MolFeat Featurizer

The main advantage of using the Molfeat library is the pre-trained featurizers, these are models provided that provide a tuned method to take a molecule from a SMILES string format and generate an embedding of the molecule that's ready to be used for downstream tasks. 
We can think of this like the pre-trained word embeddings in an NLP model.

In this case we need to use the dataset wrapper from `MolFeat` - more details of this can be found in the tutorials in the docs [here](https://molfeat-docs.datamol.io/stable/tutorials/transformer_finetuning.html).

**Exercise:** To see how this dataset class should be changed depending on the requirements of the dataset look at the PyTorch Geometric Molfeat tutorial [here](https://console.paperspace.com/github/graphcore/Gradient-Pytorch-Geometric?machine=Free-IPU-POD4&container=graphcore%2Fpytorch-geometric-jupyter%3A3.2.0-ubuntu-20.04-20230314&file=%2F%2FREADME_first.ipynb).



In [ ]:
class DTset(Dataset):
    def __init__(self, smiles, y, mf_featurizer):
        super().__init__()
        self.smiles = smiles
        self.mf_featurizer = mf_featurizer
        self.y = torch.tensor(y).float()
        # here we use the molfeat mf_featurizer to convert the smiles to
        # corresponding tokens based on the internal tokenizer
        # we just want the data from the batch encoding object
        self.transformed_mols = self.mf_featurizer._convert(smiles)

    @property
    def embedding_dim(self):
        return len(self.mf_featurizer)

    @property
    def max_length(self):
        return self.transformed_mols.shape[-1]

    def __len__(self):
        return self.y.shape[0]

    def collate_fn(self, **kwargs):
        # the default collate fn self.mf_featurizer.get_collate_fn(**kwargs)
        # returns None, which should just concatenate the inputs
        # You could also use `transformers.default_data_collator` instead
        return self.mf_featurizer.get_collate_fn(**kwargs)

    def __getitem__(self, index):
        datapoint = dict(
            (name, val[index]) for name, val in self.transformed_mols.items()
        )
        datapoint["y"] = self.y[index]
        return datapoint

## Build the poptorch Dataloader for the IPU

We need to build a Poptorch Dataloader with our dataset. We can start by processing the dataframe with the `DTset` class above - this takes the featurizer and processes all the smiles strings and returns a PyTorch dataset. 
Then after splitting the dataset into a train and test set, we can set up the IPU specific dataloader.

This takes the train split of the dataset and the `ipu_opts` to build the dataloader. 

Some key aspects of the `ipu_opts` are given below:
* `deviceIterations` - this is the number of training steps taken before the IPU communciates with the host
* `gradientAccumulation` - we can accumulate gradient updates for a number of steps before updating the weights, this can be useful for larger training pipelines, but here we set this values to 2. 
* `replciationFactor` - we can replicate the model over a number of IPUs to take advantage of data parallelism which can speed up training, however here 1x IPU is sufficient 
* `BATCH_SIZE` - this is the micro batch size, however we also have a concept of a `total batch size` that is given by `BATCH_SIZE * NUM_REPLICAS * GRADIENT_ACCUMALATION` 

More details on these can be found in the introduction tutorials - and for the models here we can safely assume that we can leave the values set here. 

In [ ]:
dataset = DTset(df.smiles.values, df.p_np.values, featurizer)

# NOTE: If you want to use the QM9 regression dataset, you need to change the args for the dataset here.
# Comment out the above line, and un-comment the below line.
# dataset = DTset(df.smiles.values, df.gap.values, featurizer)


generator = torch.Generator().manual_seed(42)
train_dt, test_dt = torch.utils.data.random_split(
    dataset, [0.8, 0.2], generator=generator
)

BATCH_SIZE = 32
# Set up the PyTorch DataLoader to load that much data at each iteration
train_opts = poptorch.Options()
train_opts.deviceIterations(1)
train_opts.Training.gradientAccumulation(2)
train_opts.replicationFactor(1)


train_loader = poptorch.DataLoader(
    options=train_opts,
    dataset=train_dt,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=dataset.collate_fn(),
)

## Model Structure

The AwesomeNet model is a custom PyTorch neural network architecture that utilizes a pre-trained transformer model from the Molfeat library as the base for molecule featurization. The architecture is comprised of several layers and components:

1. **Embedding Layer**: A copy of the base pre-trained transformer model from the MolFeat featurizer, which serves as the embedding layer for the input data.
2. **Embedding Dimension**: The size of the hidden layer in the base pre-trained transformer model.
3. **Pooling Layer**: Obtained from the MolFeat featurizer, this layer performs pooling operations on the embeddings.
4. **Hidden Layer**: A sequential layer consisting of dropout, linear, and ReLU activation layers. The input size is the length of the MolFeat featurizer, while the output size is a user-defined `hidden_size`.
5. **Output Layer**: A linear layer that maps the hidden layer's output to the final output size, which is usually set to 1 for regression tasks.
6. **Loss Function**: The model uses the Binary Cross Entropy with Logits Loss (BCEWithLogitsLoss) function to compute the loss during training.

The `forward` function of the model takes input arguments and optional target labels (y) and performs the following steps:

1. Pass the input data through the embedding layer to obtain the embeddings.
2. Extract the last hidden state from the embeddings.
3. Apply the pooling layer on the last hidden state using the input_ids and attention_mask.
4. Pass the pooled embeddings through the custom hidden layer.
5. Obtain the final output by passing the hidden layer's output through the output layer.

If target labels are provided, the model computes the loss using the BCEWithLogitsLoss function and returns the output along with the computed loss. Otherwise, only the output is returned.


In [ ]:
class AwesomeNet(torch.nn.Module):
    def __init__(self, mf_featurizer, hidden_size=128, dropout=0.1, output_size=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        # we get the underlying model from the molfeat featurizer
        # here we fetch the "base" huggingface transformer model
        # and not the wrapper around for MLM
        # this is principally to get smaller model and training efficiency
        base_pretrained_model = getattr(
            mf_featurizer.featurizer.model,
            mf_featurizer.featurizer.model.base_model_prefix,
        )
        self.embedding_layer = copy.deepcopy(base_pretrained_model)
        self.embedding_dim = mf_featurizer.featurizer.model.config.hidden_size
        # we get the the pooling layer from the molfeat featurizer
        self.pooling_layer = mf_featurizer._pooling_obj
        self.hidden_layer = torch.nn.Sequential(
            torch.nn.Dropout(p=dropout),
            torch.nn.Linear(len(mf_featurizer), self.hidden_size),
            torch.nn.ReLU(),
        )
        self.output_layer = torch.nn.Linear(self.hidden_size, self.output_size)
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

        # Swap the loss function here for a regression task
        # self.loss_fn = torch.nn.L1Loss()

    def forward(self, y=None, **kwargs):
        # get embeddings
        x = self.embedding_layer(**kwargs)
        # we take the last hidden state
        emb = x["last_hidden_state"]
        # run poolings
        h = self.pooling_layer(
            emb,
            kwargs["input_ids"],
            mask=kwargs.get("attention_mask"),
        )
        # run through our custom and optional hidden layer
        h = self.hidden_layer(h)
        # run through output layers to get logits
        if y is not None:
            out = self.output_layer(h)
            loss = self.loss_fn(out.squeeze(), y)
            return out, loss
        return self.output_layer(h)

Now we can define the training model and set some key hyoperparameters. 
* `NUM_EPOCHS` - how long to train the model for
* `LEARNING_RATE` - set the learning rate to tune the training 
* `HIDDEN_SIZE` - set the size of the final hidden layer in the model

Then we call the model and define the optimizer. 
In this case we're going to keep it simple and use Adam. 

Then the `poptorch` training model is created from our `AwesomeNet` - which we can look at in the summary below. 

In [ ]:
# Key Hyperparameters
NUM_EPOCHS = 10
LEARNING_RATE = 1e-3
HIDDEN_SZIE = 64


model = AwesomeNet(featurizer, hidden_size=HIDDEN_SZIE, dropout=0.1, output_size=1)
optimizer = poptorch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Wrap the model in a PopTorch training wrapper
train_model = poptorch.trainingModel(model, options=train_opts, optimizer=optimizer)


from torchinfo import summary

summary(train_model)

### Training the model

Now we can train the model.
This block will compile the model and loop through the dataset for the number of epochs specified. 
Finally the model is detached from the IPU to free up resources for the next step. 

In [ ]:
# Train
epoch_losses = []
from tqdm.notebook import tqdm

pbar = tqdm(range(NUM_EPOCHS), colour="#FF6F79")
for epoch in pbar:
    losses = []
    for data in train_loader:
        out, loss = train_model(**data)
        losses.append(torch.mean(loss).item())
        epoch_losses.append(torch.mean(loss).item())
    pbar.set_description(f"Epoch {epoch} - Loss {np.mean(losses):.5f}")
train_model.detachFromDevice()

And we can plot the smoothed loss history, to validate to that the training looks sensible. 


In [ ]:
from utils import plot_smoothed_loss

plot_smoothed_loss(epoch_losses)

## Evaluation

To run the evaluation we need a few key steps, these are very similar to the steps for the training. We need to define the `ipu_opts` for the test model, and we need to create a poptorch dataloader. 
1. Build the `test_loader` for the test dataset 
2. Build the poptorch inference model 
3. Loop through the test dataset to get predictions. 
    NB: Be careful with the processing of the outputs if you try the QM9 regression dataset, remove the sigmoid function on the outputs. 
4. Evaluate with the accuracy and ROC score

This will then finally print out the performance on the test dataset. 

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from matplotlib import pyplot as plt

test_opts = poptorch.Options()
test_opts.deviceIterations(1)
test_opts.replicationFactor(1)

test_loader = poptorch.DataLoader(
    options=test_opts,
    dataset=test_dt,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=dataset.collate_fn(),
)

inference_model = poptorch.inferenceModel(model, options=test_opts)
test_y_hat = []
test_y_true = []

predictions, labels = [], []
for data in test_loader:
    out, _ = inference_model(**data)
    # we apply sigmoid for classification - don't need this for regression
    out = torch.sigmoid(out)
    test_y_hat.append(out.detach().cpu().squeeze())
    test_y_true.append(data["y"])
test_y_hat = torch.cat(test_y_hat).squeeze().numpy()
test_y_true = torch.cat(test_y_true).squeeze().numpy()
assert len(test_y_true) == len(test_y_hat)
roc_auc = roc_auc_score(test_y_true, test_y_hat)
acc = accuracy_score(test_y_true, test_y_hat >= 0.5)
print(f"Test ROC AUC: {roc_auc:.3f}\nTest Accuracy: {acc:.3f}")

## Demo Molecules

We can see the accuracy and ROC score above, they seem rpetty good, but it's hard to put that in context of a real use case. 

We have a fine-tuned model, so let's use it to make predictions on individual moelcules as we loop through the dataset. 


For demonstration purposes we'll build a new data loader for inference with a abtch-size of 1 molecule, and for simplicity we'll just take the original dataset - in reality this might be a different dataset, or new molecules as they are needed to be evaluated, but this gives an idea of the workflow. 

Like previously we set up the Dataloader and ipu_opts in the same manner, and build the inference version of out model. 

In [ ]:
test_opts = poptorch.Options()
test_opts.deviceIterations(1)
test_opts.replicationFactor(1)
# Dataframe of "new molecules"
inf_df = df
dataset = DTset(inf_df.smiles.values, inf_df.p_np.values, featurizer)
# Set the batch size to 1 as we intend to process each molecule one at a time.
inf_loader = poptorch.DataLoader(
    options=test_opts,
    dataset=dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=dataset.collate_fn(),
)
# And we define the inference model here.
model.eval()
inference_model = poptorch.inferenceModel(model, options=test_opts)

Then if we turn the loader and dataframe into iterables we can loop through one molecule at a time on-demmand. 

In [ ]:
sampler = iter(inf_loader)
smiles = iter(df.smiles.values)
names = iter(df.name.values)

The following block is the inference call. The first time it is run the inference model will compile, and for subsequent calls it will run directly and can be run until the dataloader is exhuasted. 
See how the molecule report has been extended to the predicted result. 

You can re-run the next molecule cell multiple times to get a feeling for how the model performs, and what a case-by-case application feels like. 


In [ ]:
def next_molecule():

    # =============================================================
    # |                                                           |
    # |             DEMO MOLECULE PREDICTION                      |
    # |                                                           |
    # =============================================================
    clear_output(True)
    sample = next(sampler)
    name = next(names)
    smile = next(smiles)

    # Grab the true value for the report
    y_truth = sample["y"]
    sample["y"] = None

    # Run inference on the model with the single sampled molecule from the sampler
    out = inference_model(**sample)
    out = torch.sigmoid(out)

    # Report the result and plot the molecule with utility function
    view = report_molecule_classification(name, bool(y_truth), out, smile)
    view.show()

In [ ]:
# RE-RUN THIS CELL TO SEE INFERENCE RESULTS ON INDIVIDUAL MOLECULES

next_molecule()

## Conclusion

We've seen how to combine a MolFeat Featurizer and a HuggingFace Transformer to classify molecules in the BBBP dataset, and how to run the final model in a single molecule inference mode. 

**Next Steps:**
* Try tuning the hyperparameters - the length of training, the size of the hidden layer and the learning rate will all impact the final performance. 
* Try changing the dataset - the BBBP dataset is a classificiation dataset, but the code is provided in commented blocks to use the QM9 dataset for regression tasks. Start here as an extension to see how to adapt the data loading and the final layer of the model (and loss function) for the new task
* Alternatively head over to  [MoleculeNet](https://moleculenet.org/datasets-1)  and explore the datasets there - they are labeled by task so you could pick an alternative classification dataset to fine-tune on or a regression task.
* Provided in the same block as the QM9 dataset is an alternative model - `ChemGPT-4.7M`, you can try swapping the featurizer / model to finetune out and see how the performance compares, and how to tune the finetuning to achieve better down-stream results. 
 

In [ ]:
# Finally detach the inference model - this line is at the end to make sure it's not run accidentally before finishing the notebook.
inference_model.detachFromDevice()